In this homework we are going to use Jupyter/Databricks Notebooks. Please complete the tasks within one notebook. 

**TASK 1**

1.	Please run your code from previous homework, which creates a sellers_rdd, header, filter a sellers_rdd without header

In [210]:
from pyspark.sql import SparkSession
import datetime
import os
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
import glob

In [211]:
spark = SparkSession.builder.appName("Spark: Dataframes_HW").config("spark.app.name", "Spark: Dataframes_HW").getOrCreate()
sc = spark.sparkContext

In [212]:
current_directory = os.getcwd()
relative_path = os.path.join(current_directory, 'RDD_HW')
file_path = relative_path +'/sellers.csv'
sellers_rdd = sc.textFile(file_path).map(lambda x: x.split(","))
sellers_rdd_header = sellers_rdd.first()
sellers_rdd = sellers_rdd.filter(lambda x: x != sellers_rdd_header)
data = sellers_rdd.take(10)
i = 0
for item in sellers_rdd.first():
    print(sellers_rdd_header[i] ,': ', type(item))
    i += 1

seller_id :  <class 'str'>
seller_name :  <class 'str'>
daily_target :  <class 'str'>


2.	Create a DataFrame variable sellers_df_test using toDF() function.

In [213]:
sellers_df_test = sellers_rdd.toDF(["seller_id", "seller_name", "daily_target"])
display(sellers_df_test)

DataFrame[seller_id: string, seller_name: string, daily_target: string]

3.	Run the following code in order to change data types 
sellers_rdd = sellers_rdd.map(lambda x: (int(x[0]),x[1],int(x[2])))


In [214]:
sellers_rdd = sellers_rdd.map(lambda x: (int(x[0]), x[1], int(x[2])))
i = 0
for item in sellers_rdd.first():
    print(sellers_rdd_header[i] ,': ',type(item))
    i += 1

seller_id :  <class 'int'>
seller_name :  <class 'str'>
daily_target :  <class 'int'>


4.	Recreate a DataFrame variable sellers_df using createDataFrame() function. For this task you need to create a schema. Review the dataset in order to choose correct data types.

In [215]:
schema = StructType([
    StructField("seller_id", IntegerType(), True),
    StructField("seller_name", StringType(), True),
    StructField("daily_target", IntegerType(), True)
])
sellers_df = spark.createDataFrame(sellers_rdd, schema=schema)
display(sellers_df)

DataFrame[seller_id: int, seller_name: string, daily_target: int]

5.	Create a Pandas dataframe sellers_df_pd  from Spark dataframe.

In [216]:
sellers_df_pd  = sellers_df.toPandas()
print(sellers_df_pd)

   seller_id seller_name  daily_target
0          0    seller_0        100000
1          1    seller_1         83478
2          2    seller_2         94114
3          3    seller_3         50299
4          4    seller_4         72654
5          5    seller_5         28862
6          6    seller_6         61878
7          7    seller_7         72047
8          8    seller_8         54715
9          9    seller_9         82824


6.	Take first 5 rows of sellers_df

In [217]:
first_5_rows = sellers_df.head(5)
for row in first_5_rows:
    print(row)

Row(seller_id=0, seller_name='seller_0', daily_target=100000)
Row(seller_id=1, seller_name='seller_1', daily_target=83478)
Row(seller_id=2, seller_name='seller_2', daily_target=94114)
Row(seller_id=3, seller_name='seller_3', daily_target=50299)
Row(seller_id=4, seller_name='seller_4', daily_target=72654)


7.	Return a schema and dtypes of sellers_df.

In [218]:
print("Schema of sellers_df:")
sellers_df.printSchema()
print("Data Types of columns in sellers_df:")
for field, data_type in sellers_df.dtypes:
    print(f"Column '{field}' has data type '{data_type}'")

Schema of sellers_df:
root
 |-- seller_id: integer (nullable = true)
 |-- seller_name: string (nullable = true)
 |-- daily_target: integer (nullable = true)

Data Types of columns in sellers_df:
Column 'seller_id' has data type 'int'
Column 'seller_name' has data type 'string'
Column 'daily_target' has data type 'int'


**TASK 2**

1.	Upload data from products.csv to products_df Dataframe variable and sales.csv to sales_df using the spark.read.csv() function. Please take a look on the header parameter while data loading.

In [219]:
products_df = spark.read.csv(os.path.join(relative_path, 'products.csv'), header=True)
sales_df = spark.read.csv(os.path.join(relative_path, 'sales.csv'), header=True)

2.	Show the content of products_df

In [220]:
products_df.show()

+----------+------------+-----+
|product_id|product_name|price|
+----------+------------+-----+
|         0|   product_0|   22|
|         1|   product_1|   35|
|         2|   product_2|  146|
|         3|   product_3|   17|
|         4|   product_4|   66|
|         5|   product_5|   31|
|         6|   product_6|  127|
|         7|   product_7|  116|
|         8|   product_8|  121|
|         9|   product_9|   98|
|        10|  product_10|   54|
|        11|  product_11|   25|
|        12|  product_12|  125|
|        13|  product_13|    8|
|        14|  product_14|  100|
|        15|  product_15|  111|
|        16|  product_16|    1|
|        17|  product_17|  115|
|        18|  product_18|   69|
|        19|  product_19|   59|
+----------+------------+-----+
only showing top 20 rows



3.	Show the summary statistic of products_df

In [221]:
products_df.describe().show()

+-------+-----------------+-------------+-----------------+
|summary|       product_id| product_name|            price|
+-------+-----------------+-------------+-----------------+
|  count|           100000|       100000|           100000|
|   mean|          49999.5|         null|         75.60715|
| stddev|28867.65779668774|         null|43.33316001060901|
|    min|                0|    product_0|                1|
|    max|            99999|product_99999|               99|
+-------+-----------------+-------------+-----------------+



4.	Print the physical and logical plans of products_df

In [222]:
print("Physical Plan:")
products_df.explain()
print("\nLogical Plan and Physical Plan:")
products_df.explain(True)

Physical Plan:
== Physical Plan ==
FileScan csv [product_id#2244,product_name#2245,price#2246] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/RDD_HW/products.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<product_id:string,product_name:string,price:string>



Logical Plan and Physical Plan:
== Parsed Logical Plan ==
Relation [product_id#2244,product_name#2245,price#2246] csv

== Analyzed Logical Plan ==
product_id: string, product_name: string, price: string
Relation [product_id#2244,product_name#2245,price#2246] csv

== Optimized Logical Plan ==
Relation [product_id#2244,product_name#2245,price#2246] csv

== Physical Plan ==
FileScan csv [product_id#2244,product_name#2245,price#2246] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/RDD_HW/products.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<product_id:string,product_name:string,price:string

5.	Count a number of distinct rows of products_df

In [223]:
print('Count a number of distinct rows of products_df:',products_df.distinct().count())

Count a number of distinct rows of products_df: 100000


6.	Transform a dataframe to RDD products_rdd variable using products_df.rdd method and print a number of partitions

In [224]:
products_rdd = products_df.rdd
num_partitions = products_rdd.getNumPartitions()
print("Number of partitions in products_rdd:", num_partitions)

Number of partitions in products_rdd: 1


7.	Filter products_df where price > 100 and save a dataframe to the products_df_more_than_100.csv file. 

In [225]:
filtered_products_df = products_df.filter(products_df["price"] > 100)
output_file_name = "products_df_more_than_100"
output_file_path = os.path.join(relative_path, output_file_name)
filtered_products_df.coalesce(1).write.format("com.databricks.spark.csv").mode("overwrite").option("header", "true").csv(output_file_path)

In [226]:
# Specify the file pattern to match
file_pattern = 'part*.csv'
wildcard_pattern = os.path.join(output_file_path, file_pattern)
# Use glob to find matching files
matching_files = glob.glob(wildcard_pattern)
# rename csv file
if len(matching_files) == 1:
    new_file_path = os.path.join(output_file_path, 'products_df_more_than_100.csv')
    os.rename(matching_files[0], new_file_path)
else:
    print('Something goes wrong, result possibly saved more than in 1 file or result did non save')

8.	Save products_df to JSON and Parquet files.

In [227]:
json_output_file_path = relative_path+"/products_df_json"
parquet_output_file_path = relative_path+"/products_df_parquet"
products_df.write.mode("overwrite").json(json_output_file_path)
products_df.write.mode("overwrite").parquet(parquet_output_file_path)

9.	Upload the JSON file from the step above to the products_df_from_json Dataframe variable and show the content of it.

In [228]:
products_df_from_json = spark.read.json(json_output_file_path)
products_df_from_json.show()

+-----+----------+------------+
|price|product_id|product_name|
+-----+----------+------------+
|   22|         0|   product_0|
|   35|         1|   product_1|
|  146|         2|   product_2|
|   17|         3|   product_3|
|   66|         4|   product_4|
|   31|         5|   product_5|
|  127|         6|   product_6|
|  116|         7|   product_7|
|  121|         8|   product_8|
|   98|         9|   product_9|
|   54|        10|  product_10|
|   25|        11|  product_11|
|  125|        12|  product_12|
|    8|        13|  product_13|
|  100|        14|  product_14|
|  111|        15|  product_15|
|    1|        16|  product_16|
|  115|        17|  product_17|
|   69|        18|  product_18|
|   59|        19|  product_19|
+-----+----------+------------+
only showing top 20 rows



In [229]:
spark.stop()